# Sentinel-2 Cloud native processing workflow

## Goal 

Prepare the CWL Workflow orchestrating CWL CommandLineTool document(s) wrapping the command line tool available container(s).

This notebook is linked to: 
https://eoap.github.io/mastering-app-package/cwl-workflow/cloud-native/


The Cloud native Workflow chains the `crop`, `norm_diff`, `otsu` and `stac` steps taking a single STAC item as input parameters:

- a SpatioTemporal Asset Catalog (STAC) Item
- a bounding box area of interest (AOI)
- the EPSG code of the bounding box area of interest
- a list of common band names (["`green`", "`nir`"])

## Setup

In [1]:
export WORKSPACE=/workspace/mastering-app-package
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

## CWL Workflow

We now have a `$graph` list and several CWL descriptions: one `Workflow` and four `CommandLineTool`:

In [8]:
cat ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[].class' -

Workflow
CommandLineTool
CommandLineTool
CommandLineTool
CommandLineTool


The `CommandLineTool` ids are the all the CommandLineTool created in the previous step:

In [14]:
cat ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[1].id' -
cat ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[2].id' -
cat ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[3].id' -
cat ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[4].id' -

crop
norm_diff
otsu
stac


Let's look at the `Workflow`:


In [16]:
cat ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0]' -

class: Workflow
id: main
label: Water bodies detection based on NDWI and the otsu threshold
doc: Water bodies detection based on NDWI and otsu threshold applied to a single Sentinel-2 COG STAC item
requirements:
  - class: ScatterFeatureRequirement
inputs:
  aoi:
    label: area of interest
    doc: area of interest as a bounding box
    type: string
  epsg:
    label: EPSG code
    doc: EPSG code
    type: string
    default: "EPSG:4326"
  bands:
    label: bands used for the NDWI
    doc: bands used for the NDWI
    type: string[]
    default: ["green", "nir"]
  item:
    doc: Reference to a STAC item
    label: STAC item reference
    type: string
outputs:
  - id: stac_catalog
    outputSource:
      - node_stac/stac_catalog
    type: Directory
steps:
  node_crop:
    run: "#crop"
    in:
      item: item
      aoi: aoi
      epsg: epsg
      band: bands
    out:
      - cropped
    scatter: band
    scatterMethod: dotproduct
  node_normalized_difference:
    run: "#norm_diff"
    i

Let's look at the `inputs` element.

These are the Application Package inputs: 

- a SpatioTemporal Asset Catalog (STAC) Item: `item`
- a bounding box area of interest (AOI): `aoi`
- the EPSG code of the bounding box area of interest: `epsg`
- a list of common band names (["`green`", "`nir`"]): `bands`

In [18]:
cat ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0].inputs' -

aoi:
  label: area of interest
  doc: area of interest as a bounding box
  type: string
epsg:
  label: EPSG code
  doc: EPSG code
  type: string
  default: "EPSG:4326"
bands:
  label: bands used for the NDWI
  doc: bands used for the NDWI
  type: string[]
  default: ["green", "nir"]
item:
  doc: Reference to a STAC item
  label: STAC item reference
  type: string


They all have an `id`, a `label` and a `doc` that describes them

Let's look at the `outputs` element.

The output is a STAC catalog, output id `stac_catalog` and its source comes from `node_stac`. 

`node_stac` is the last step of the `Workflow`


In [20]:
cat ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0].outputs' -

- id: stac_catalog
  outputSource:
    - node_stac/stac_catalog
  type: Directory


Let's look at the Workflow steps.

It is a mapping of `steps`, each with inputs `in`, outputs `out` and a CWL to run `run` that uses an anchor to the `CommandLineTool` id: 

In [22]:
cat ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0].steps' -

node_crop:
  run: "#crop"
  in:
    item: item
    aoi: aoi
    epsg: epsg
    band: bands
  out:
    - cropped
  scatter: band
  scatterMethod: dotproduct
node_normalized_difference:
  run: "#norm_diff"
  in:
    rasters:
      source: node_crop/cropped
  out:
    - ndwi
node_otsu:
  run: "#otsu"
  in:
    raster:
      source: node_normalized_difference/ndwi
  out:
    - binary_mask_item
node_stac:
  run: "#stac"
  in:
    item: item
    rasters:
      source: node_otsu/binary_mask_item
  out:
    - stac_catalog


The first step, `crop`, applies the fan-out pattern on the input `bands` which is a list. 

This step is invoked with `bands: [green, nir]`

The `in` mapping maps the step inputs to the Workflow inputs.

In [24]:
cat ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0].steps["node_crop"]' -

run: "#crop"
in:
  item: item
  aoi: aoi
  epsg: epsg
  band: bands
out:
  - cropped
scatter: band
scatterMethod: dotproduct


The second step applies the normalized difference.

The `in` mapping defines the `rasters` inputs as the outputs of the `node_crop` step with `source: node_crop/cropped`. This is how the orchestration of the steps is defined in CWL.



In [26]:
cat ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0].steps["node_normalized_difference"]' -

run: "#norm_diff"
in:
  rasters:
    source: node_crop/cropped
out:
  - ndwi


The third step behaves the same way:

In [28]:
cat ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0].steps["node_otsu"]' -

run: "#otsu"
in:
  raster:
    source: node_normalized_difference/ndwi
out:
  - binary_mask_item


The final step, `node_stac` takes the Workflow input `stac_item` and the output of `node_otsu`.

In [29]:
cat ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0].steps["node_stac"]' -

run: "#stac"
in:
  item: item
  rasters:
    source: node_otsu/binary_mask_item
out:
  - stac_catalog


## Running the Workflow

In [31]:
cwltool \
    --parallel \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-workflow/app-water-body-cloud-native.cwl \
    --item "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A" \
    --aoi="-121.399,39.834,-120.74,40.472" \
    --epsg "EPSG:4326" \
    --bands green \
    --bands nir

INFO /home/fbrito/.local/bin/cwltool 3.1.20240112164112
INFO Resolved '/workspace/mastering-app-package/cwl-workflow/app-water-body-cloud-native.cwl' to 'file:///workspace/mastering-app-package/cwl-workflow/app-water-body-cloud-native.cwl'
INFO [workflow ] starting step node_crop
INFO [step node_crop] start
INFO [workflow ] start
INFO [step node_crop] start
WARNING [job node_crop_2] Skipping Docker software container '--memory' limit despite presence of ResourceRequirement with ramMin and/or ramMax setting. Consider running with --strict-memory-limit for increased portability assurance.
WARNING [job node_crop_2] Skipping Docker software container '--cpus' limit despite presence of ResourceRequirement with coresMin and/or coresMax setting. Consider running with --strict-cpu-limit for increased portability assurance.
INFO [job node_crop_2] /tmp/2d7t1zi9$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/2d7t1zi9,target=/XlYjPs \
    --mount=type=bind,so